In [44]:
## fetching the data from API

import urllib.request as request
import json
with request.urlopen('https://data.cdc.gov/resource/mqmc-4b9n.json?$limit=20000') as response:
        if response.getcode() == 200:
            source = response.read()
            data = json.loads(source)
        else:
            print('An error occurred while attempting to retrieve data from the API.')

In [45]:
## Connection strings for MongoDB Atlas cloud and passing the user name and password from configuration file
import pymongo
from  pymongo import MongoClient
import urllib.parse
from config import manish_mongo_config

mongo_parameters=manish_mongo_config()
client = pymongo.MongoClient(mongo_parameters['url'])
print(client.list_database_names())

##Checking the database condition if the database already exist the

Database=client["Team_F_COVID19USA"]

db_name = "Team_F_COVID19USA"
if db_name in client.list_database_names():
    print("Database Exist")
else:
    print("Creating the Database again")
    Database=client["Team_F_COVID19USA"]
        
##check if collection with name "CovidImpact" exit if yes then drop and Recreate it,  if not exist then create it.    


if  ("Covid19Analysis" in Database.list_collection_names()):
    collection = Database['Covid19Analysis']
    print("Collection exist")
    Collection=Database["Covid19Analysis"]
    
else:
    
    print("Creating a collection again")
    Collection=Database["Covid19Analysis"]
    Collection.insert_many(data)

        

['Team_F_COVID19USA', 'admin', 'local']
Database Exist
Collection exist


In [46]:
##Fetching  the data from MongoDB Atlas and creating a dataframe

import pandas as pd

Analyis=pd.DataFrame(Collection.find())

In [47]:
print(Analyis.head(10))

                        _id               data_as_of               start_date  \
0  6084750f95a27c2a8b8fbe57  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   
1  6084750f95a27c2a8b8fbe58  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   
2  6084750f95a27c2a8b8fbe59  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   
3  6084750f95a27c2a8b8fbe5a  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   
4  6084750f95a27c2a8b8fbe5b  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   
5  6084750f95a27c2a8b8fbe5c  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   
6  6084750f95a27c2a8b8fbe5d  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   
7  6084750f95a27c2a8b8fbe5e  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   
8  6084750f95a27c2a8b8fbe5f  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   
9  6084750f95a27c2a8b8fbe60  2021-04-18T00:00:00.000  2015-01-10T00:00:00.000   

                  end_date mmwr_year mmwr_week         week_ending_date  \
0  2021-02-27T00:00:00.000      2

In [48]:
# Removing unwanted columns

UpdatedAnalyis = Analyis.loc[:, ~Analyis.columns.isin(['data_as_of','_id','end_date','mmwr_year','hrr_number','mmwr_week','week_ending_date','footnote'])]

print(UpdatedAnalyis)

                    start_date       hrr_name state      total_deaths  \
0      2015-01-10T00:00:00.000  AK- ANCHORAGE    AK  15.1953378548669   
1      2015-01-10T00:00:00.000  AK- ANCHORAGE    AK  16.1103063651122   
2      2015-01-10T00:00:00.000  AK- ANCHORAGE    AK  21.4528695593406   
3      2015-01-10T00:00:00.000  AK- ANCHORAGE    AK  19.6028038764853   
4      2015-01-10T00:00:00.000  AK- ANCHORAGE    AK   16.859181027018   
...                        ...            ...   ...               ...   
19995  2015-01-10T00:00:00.000    FL- ORLANDO    FL  114.591007923336   
19996  2015-01-10T00:00:00.000    FL- ORLANDO    FL  99.2603397230077   
19997  2015-01-10T00:00:00.000    FL- ORLANDO    FL  114.309087967336   
19998  2015-01-10T00:00:00.000    FL- ORLANDO    FL  112.346044668542   
19999  2015-01-10T00:00:00.000    FL- ORLANDO    FL  115.970805666633   

      covid_19_deaths total_deaths_over_65_years  \
0                   0           8.38949440393337   
1                  

In [49]:
## Checking Null values in the dataframe 

UpdatedAnalyis.isnull().sum()

start_date                        0
hrr_name                          0
state                             0
total_deaths                      0
covid_19_deaths                  79
total_deaths_over_65_years        0
covid_19_deaths_over_65_years    72
accuracy_index                    0
dtype: int64

In [50]:

## usking sklearn for  imputing missing values

## SettingWithCopyWarning supression

pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(UpdatedAnalyis.iloc[:, 4:7])
UpdatedAnalyis.iloc[:, 4:7] = imputer.transform(UpdatedAnalyis.iloc[:, 4:7])


In [51]:
## checking again Null values after applying missing imputation

UpdatedAnalyis.isnull().sum()

start_date                       0
hrr_name                         0
state                            0
total_deaths                     0
covid_19_deaths                  0
total_deaths_over_65_years       0
covid_19_deaths_over_65_years    0
accuracy_index                   0
dtype: int64

In [52]:
##Identifying Total deaths over the age of 65 years by grouping by clause in state

## Top 5 state with most no of Total death 

Covi=UpdatedAnalyis.groupby(['state',],as_index=False).agg({'total_deaths':max,'total_deaths_over_65_years':max})

Stateanalyis =Covi.sort_values(by ='total_deaths', ascending=False)
Stateanalyis.head(5)


,state,total_deaths,total_deaths_over_65_years
4,CA,999.409513556105,2708.795900
9,FL,99.9585912450061,531.577620
8,DE,99.9083472867204,92.729857
5,CO,99.7440168068817,114.268824
7,DC,99.575724119172,134.679146


In [53]:
## Bar Graph represnting Total deaths and death of the people over aged 65

import plotly.graph_objs as go
from plotly.offline import iplot
Graph1  = go.Bar(
                x = Covi.state,
                y = Covi.total_deaths_over_65_years,
                name = "total_deaths_over_65_years",
                marker = dict(color = 'cornflowerblue',
                             line=dict(color='cornflowerblue',width=1.5)),
                text = Covi.state)



Graph2 = go.Bar(
                x = Covi.state,
                y = Covi.total_deaths,
                name = "total_deaths",
                marker = dict(color = 'lightblue',
                              line=dict(color='lightblue',width=1.5)),
                text = Covi.state)
data = [Graph1 , Graph2 ]
layout = {
  'xaxis': {'title': 'Different U.S. States'},
  'barmode': 'relative',
  'title': 'Comparison between Total Death Count For People Over 65 years age v.s. Total Deaths Among All'};
fig = go.Figure(data = data, layout = layout)
iplot(fig)


In [54]:
## Top 5 state with Total death due to most number of  covid death over 65 years of age

Death_age= UpdatedAnalyis.groupby(['state'],as_index=False).agg({'covid_19_deaths_over_65_years':max,'covid_19_deaths':max})
cases = Death_age.sort_values(by ='covid_19_deaths', ascending=False)
cases.head(5)


,state,covid_19_deaths_over_65_years,covid_19_deaths
4,CA,1359.287948,1918.647669
3,AZ,465.062793,611.541354
9,FL,238.051468,303.883596
6,CT,225.006905,250.009994
7,DC,51.936647,63.299285


In [55]:
## Pie chart reprenting deaths due to Covid 19 aged more then 65

import plotly.express as px
import plotly.io as pio

chart = px.pie(
        data_frame=Death_age,
        values='covid_19_deaths_over_65_years',
        names='state',
        color='state',                      
                
               
        labels={"state":"State"},     
        title='U.S. States With Covid19deaths over 65 years',     
        template='presentation',            
                                            
                                           
        width=700,                          
        height=500,                         
                              
        )

chart.update_traces(textposition='inside', textinfo='percent+label',
                        marker=dict(line=dict(color='#000000', width=4)),
                        pull=[0, 0, 0, 0], opacity=0.7, rotation=180)



pio.show(chart)

In [56]:
## Top 5 state by covid 19 deaths over 65 years of age

Death_updated= UpdatedAnalyis.groupby(['state'],as_index=False).agg({'total_deaths':max,'covid_19_deaths':max,'covid_19_deaths_over_65_years':max})
Coviddeath = Death_updated.sort_values(by ='covid_19_deaths_over_65_years', ascending=False)

Coviddeath.head(5)

,state,total_deaths,covid_19_deaths,covid_19_deaths_over_65_years
4,CA,999.409513556105,1918.647669,1359.287948
3,AZ,986.646157815725,611.541354,465.062793
9,FL,99.9585912450061,303.883596,238.051468
6,CT,99.5738304432262,250.009994,225.006905
5,CO,99.7440168068817,62.580634,52.111532


In [57]:
## creating a heat map shping Death and Total deaths

import plotly.express as px
fig = px.density_heatmap(Death_updated, x="covid_19_deaths", y="total_deaths",
      title="Heatmap Demonstrating Total death count and Covid19 in USA ")
fig.show()

In [58]:
## top 5 state with the most number of  total deaths ,covid19 death and total death over 65 years of age

Age_effect= UpdatedAnalyis.groupby(['state'],as_index=False).agg({'total_deaths':max,'covid_19_deaths':max,'total_deaths_over_65_years':max})

Deathcount =Age_effect.sort_values(by ='total_deaths_over_65_years', ascending=False)

Deathcount.head(5)


,state,total_deaths,covid_19_deaths,total_deaths_over_65_years
4,CA,999.409513556105,1918.647669,2708.795900
3,AZ,986.646157815725,611.541354,1097.533121
9,FL,99.9585912450061,303.883596,531.577620
6,CT,99.5738304432262,250.009994,369.014355
7,DC,99.575724119172,63.299285,134.679146


In [59]:
## creating a scatter plot represnting the covid_19_deaths and total_deaths_over_65_years among various states

import plotly.express as px
fig = px.scatter(Age_effect, x="covid_19_deaths", y="total_deaths_over_65_years", color="state",
      title="Scatter Plot showing People died with age more than 65 due to Covid19 ")
fig.show()

In [60]:
##Importing psycopg2 library
import psycopg2


##Importing config parameters from config.py
from config import config

 
## feteching the config file parameters
params_ = config()

 ## Connect to the Postgres_DB:
Postgre_Connection = psycopg2.connect(**params_)
Postgre_Connection.autocommit=True
## Create cursor allowing us execute PSQL using Python:
cursor = Postgre_Connection.cursor()

In [61]:
##Importing alchemy_config  from config.py file.
from config import alchemy_config

##Importing the library sqlalchemy
import sqlalchemy

##Creating object for alchemy_config:
sqlalch=alchemy_config()

In [62]:
## checking if the database 'team_f_covid_data' exist or not and recreating if it does not exist

cursor.execute("SELECT datname FROM pg_database;")
list_database = cursor.fetchall()
database_name = 'team_f_covid_data'

if (database_name,) in list_database:
    print("'{}' Database already exist".format(database_name))
else:
    print("'{}' Database not exist.".format(database_name))
    cursor.execute ('CREATE database team_f_covid_data;') 
    print("Database has been created ")
    cursor.close()
 

'team_f_covid_data' Database already exist


In [63]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'ORDER BY table_name;")
fetch_tables = cursor.fetchall()
Table_name='dataset2_manish'

##Fething the data from loop for checking tables "dataset2_manish" existence 

for k in fetch_tables:
    if("dataset2_manish" in k ):
        print("'{}' Table already exist".format(Table_name))
        break
## If the table 'dataset2_manish' does not exixt  recreate it 
else:
    print("'{}' Table not exist.".format(Table_name))
    import sqlalchemy
    engine=sqlalchemy.create_engine(sqlalch['alchemyengine'])
    UpdatedAnalyis.to_sql('dataset2_manish',engine,if_exists='append')
    print("Table has been created ")

## closing the cursor
cursor.close()
print("Cursor Closed")

'dataset2_manish' Table not exist.
Table has been created 
Cursor Closed
